This is my THIRD attempt at making this model.In the first attempt i did not use BERT

#Attempt 1
normal :-
 MAE: 814.313290713805                 
 ---------------RMSE: 3284.8791334935618

#Attempt 2
used log1p(likes):- MAE: 541.7342766493772              
--------------------RMSE: 2953.6572770854145

#Attempt 3
using bert and log of lieks:- MAE: 602.95                  
--------------------------------------------RMSE: 3177.70

In [3]:
!pip install -q sentence-transformers


import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt

In [4]:
from google.colab import files
uploaded = files.upload()

Saving behaviour_simulation_train.xlsx - Sheet1 (1).csv to behaviour_simulation_train.xlsx - Sheet1 (1).csv


In [5]:
import pandas as pd

# Load the CSV file (replace with your actual file path)
df = pd.read_csv("behaviour_simulation_train.xlsx - Sheet1 (1).csv")  # replace with your actual path

# Basic data inspection
print(df.shape)         # Shows number of rows and columns
print(df.head())        # Displays the first 5 rows
print(df.info())        # Provides summary info of the DataFrame
print(df.isnull().sum())  # Counts missing values in each column

(17331, 7)
   id                 date  likes  \
0   1  2020-12-12 00:47:00      1   
1   2  2018-06-30 10:04:20   2750   
2   3  2020-09-29 19:47:28     57   
3   4  2020-10-01 11:40:09    152   
4   5  2018-10-19 14:30:46     41   

                                             content        username  \
0  Spend your weekend morning with a Ham, Egg, an...    TimHortonsPH   
1  Watch rapper <mention> freestyle for over an H...       IndyMusic   
2  Canadian Armenian community demands ban on mil...       CBCCanada   
3  1st in Europe to be devastated by COVID-19, It...  MKWilliamsRome   
4  Congratulations to Pauletha Butts of <mention>...           BGISD   

                                               media inferred company  
0  [Photo(previewUrl='https://pbs.twimg.com/media...      tim hortons  
1  [Photo(previewUrl='https://pbs.twimg.com/media...      independent  
2  [Photo(previewUrl='https://pbs.twimg.com/media...              cbc  
3  [Photo(previewUrl='https://pbs.twimg.com/m

In [6]:
df.dropna(subset=['content', 'username', 'inferred company', 'likes'], inplace=True)
df['media'].fillna('no_media', inplace=True)
df['has_media'] = df['media'].apply(lambda x: x != 'no_media')
df['content'] = df['content'].astype(str).str.strip().str.lower()
df['datetime'] = pd.to_datetime(df['date'], errors='coerce')

<ipython-input-6-58ea6ea02cfc>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['media'].fillna('no_media', inplace=True)


In [13]:
!pip install emoji


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 8.1 MB/s eta 0:00:00


In [14]:
import emoji
import re

#  I counted number of emojis in the tweet
def count_emojis(text):
    return sum(1 for char in str(text) if char in emoji.EMOJI_DATA)

# then checked the hastags
def has_hashtag(text):
    return 1 if re.search(r"#\w+", str(text)) else 0


df['emoji_count'] = df['content'].apply(count_emojis)
df['has_hashtag'] = df['content'].apply(has_hashtag)


In [7]:
#  Adding Sentiment
!pip install -q textblob
from textblob import TextBlob
df['sentiment'] = df['content'].astype(str).apply(lambda x: TextBlob(x).sentiment.polarity)

In [9]:
df['word_count'] = df['content'].apply(lambda x: len(str(x).split()))
df['char_count'] = df['content'].apply(lambda x: len(str(x)))
df['has_media'] = df['media'].apply(lambda x: 1 if x else 0)

df['datetime'] = pd.to_datetime(df['datetime'])
df['hour'] = df['datetime'].dt.hour
df['day_of_week'] = df['datetime'].dt.dayofweek


df['company_encoded'] = LabelEncoder().fit_transform(df['inferred company'])
df['username_encoded'] = LabelEncoder().fit_transform(df['username'])

In [10]:
print("Generating BERT embeddings...")
model = SentenceTransformer('all-MiniLM-L6-v2')
bert_embeddings = model.encode(df['content'].astype(str).tolist(), show_progress_bar=True)


bert_df = pd.DataFrame(bert_embeddings, columns=[f'bert_{i}' for i in range(bert_embeddings.shape[1])])
df = pd.concat([df.reset_index(drop=True), bert_df], axis=1)

Generating BERT embeddings...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/542 [00:00<?, ?it/s]

In [15]:
basic_features = ['sentiment', 'has_media', 'hour', 'day_of_week',
                  'company_encoded', 'username_encoded', 'emoji_count', 'has_hashtag']

bert_features = [f'bert_{i}' for i in range(bert_embeddings.shape[1])]
features = basic_features + bert_features

X = df[features]
y = np.log1p(df['likes'])  # log1p

In [16]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Model Training
model = RandomForestRegressor()
model.fit(X_train, y_train)

RandomForestRegressor()

In [18]:
# Prediction
y_pred_log = model.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))

print(f"\n📊 Model with BERT Embeddings:")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")


📊 Model with BERT Embeddings:
MAE: 602.95
RMSE: 3177.70


#next everything is chatgpt
i did not get enough time to read thru the resourses so i just asked chatgpt

In [20]:
import joblib
joblib.dump(model, 'like_predictor.pkl')


['like_predictor.pkl']

In [21]:
!pip install flask-ngrok


In [ ]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import joblib
import numpy as np

# Load the model
model = joblib.load('like_predictor.pkl')

# Create Flask app
app = Flask(__name__)
run_with_ngrok(app)  # starts ngrok when app is run

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json()

    # Adjust features here based on your model
    features = np.array([
        data['word_count'],
        data['char_count'],
        data['sentiment'],
        data['has_media'],
        data['hour'],
        data['day_of_week'],
        data['company_encoded'],
        data['username_encoded'],
        data['emoji_count'],
        data['has_hashtag']
    ]).reshape(1, -1)

    prediction = model.predict(features)[0]
    return jsonify({'predicted_likes': int(prediction)})

# Start Flask app
app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-11:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
           